In [1]:
print("Importa librerie", end="", flush=True)
from keras_tuner import HyperModel, RandomSearch, Hyperband, BayesianOptimization
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, RepeatVector, Dropout, TimeDistributed, Dense
print(".", end="", flush=True)
from tensorflow.keras.models import Sequential
print(".", end="", flush=True)
from tensorflow.keras.regularizers import l2
print(".", end="", flush=True)
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from sklearn.preprocessing import PowerTransformer
print(".", end="", flush=True)
from sklearn.base import clone
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)


Importa librerie...........=


In [2]:
scaler = PowerTransformer()
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = 1000
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo

elenco_features = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "Volume",
    "MACDh",
    "PSAR",
    "PSARaf",
    "SUPERT", 
    "TRIX",
    "ATR",
    "DM_OSC",
    "ADX",
    "CMF"
]
elenco_target = [
    "EMA_5",
    "EMA_20", 
    "EMA_50"    
]

col_features = {col: idx for idx, col in enumerate(elenco_features)}
col_targets = {col: idx for idx, col in enumerate(elenco_target)}
n_features = len(col_features)
n_targets = len(col_targets)

class MixedHyperModel(HyperModel):
    def __init__(self, n_timesteps, n_features, n_targets, giorni_previsione):
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.n_targets = n_targets
        self.giorni_previsione = giorni_previsione

    def build(self, hp):
        model = Sequential()

        # Layer LSTM iniziale
        model.add(LSTM(hp.Int('lstm_units_1', 10, 150, step=10),
                       input_shape=(self.n_timesteps, self.n_features)))
        model.add(RepeatVector(self.giorni_previsione))

        # Aggiunta di layer LSTM intermedi
        for i in range(hp.Int('num_lstm_layers', 1, 3)):
            model.add(LSTM(hp.Int(f'lstm_units_{i+2}', 10, 150, step=10), return_sequences=True,
                           kernel_regularizer=l2(hp.Float('l2_rate', 1e-5, 1e-1, sampling='log'))))
            model.add(Dropout(hp.Float(f'lstm_dropout_{i+2}', 0, 0.5, step=0.1)))

        # Aggiunta di layer Dense
        for i in range(hp.Int('num_dense_layers', 1, 3)):
            model.add(TimeDistributed(Dense(hp.Int(f'dense_units_{i}', 10, 150, step=10), activation='relu')))
            model.add(Dropout(hp.Float(f'dense_dropout_{i}', 0, 0.5, step=0.1)))

        model.add(TimeDistributed(Dense(self.n_targets, activation='linear')))

        model.compile(optimizer=optimizer,
                      loss='mean_absolute_percentage_error')

        return model

input_shape = (n_timesteps, n_features)
output_shape = (giorni_previsione, n_targets)
hypermodel = MixedHyperModel(n_timesteps, n_features, n_targets, giorni_previsione)

print("Download lista ticker")
lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]

print("Download dati ticker validazione")
ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
ticker_val.index = ticker_val.index.date
print("Calcolo indicatori ticker validazione")
ticker_val = fx.crea_indicatori(ticker_val)
ticker_val.dropna(axis=0, inplace=True)

idx_val, X_val, Y_val = fx.to_XY(ticker_val, elenco_features, elenco_target, n_timesteps, giorni_previsione, addestramento=True)
X_val = X_val.reshape((-1, n_timesteps * n_features))
Y_val = Y_val.reshape((-1, giorni_previsione * n_targets))
X_scaler_val = clone(scaler)
Y_scaler_val = clone(scaler)
X_val = X_scaler_val.fit_transform(X_val)
Y_val = Y_scaler_val.fit_transform(Y_val)
X_val = X_val.reshape((-1, n_timesteps, n_features))
Y_val = Y_val.reshape((-1, giorni_previsione, n_targets))

if os.path.exists("X.npy") and os.path.exists("Y.npy"):
    X = np.load('X.npy')
    Y = np.load('Y.npy')
else:
    X = np.zeros((0, n_timesteps, n_features))
    Y = np.zeros((0, giorni_previsione, n_targets))
    for i_ticker in range (n_simboli_addestramento):
        nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
        print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
        print("Download dati ticker")
        try:
            ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
            if ticker["Close"].iloc[-1] >= 1:
                ticker.index = ticker.index.date
                print("Calcolo indicatori ticker")
                ticker = fx.crea_indicatori(ticker)
                ticker.dropna(axis=0, inplace=True)

                print("Definizione features e target")
                idx, X, Y = fx.to_XY(ticker, elenco_features, elenco_target, n_timesteps, giorni_previsione, addestramento=True)

                print("Scaler")
                X_train = X.reshape((-1, n_timesteps * n_features))
                Y_train = Y.reshape((-1, giorni_previsione * n_targets))
                X_scaler = clone(scaler)
                Y_scaler = clone(scaler)
                X_train = X_scaler.fit_transform(X_train)
                Y_train = Y_scaler.fit_transform(Y_train)
                X_train = X_train.reshape((-1, n_timesteps, n_features))
                Y_train = Y_train.reshape((-1, giorni_previsione, n_targets))

                X = np.vstack((X, X_train))        
                Y = np.vstack((Y, Y_train))
        except Exception as e:
            print(e)
            continue
    print("Salvataggio X e Y su file")
    np.save('X', X)
    np.save('Y', Y)

Download lista ticker
Download dati ticker validazione
Calcolo indicatori ticker validazione
1 di 500: Ticker MTDR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
2 di 500: Ticker KMB
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
3 di 500: Ticker KRTX
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
4 di 500: Ticker SNA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
5 di 500: Ticker CPF
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
6 di 500: Ticker DTC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
7 di 500: Ticker BOX
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
8 di 500: Ticker DDOG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
9 di 500: Ticker PRG
Download dati ticker
Calcolo indicatori tic


1 Failed download:
['UHAL^B']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
69 di 500: Ticker BOH
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
70 di 500: Ticker TTWO
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
71 di 500: Ticker LUV
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
72 di 500: Ticker FRPH
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
73 di 500: Ticker LPLA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
74 di 500: Ticker GLPG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
75 di 500: Ticker FELE
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
76 di 500: Ticker 1NOKIA
Download dati ticker



1 Failed download:
['1NOKIA']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
77 di 500: Ticker REX
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
78 di 500: Ticker KMPR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
79 di 500: Ticker OKTA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
80 di 500: Ticker G
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
81 di 500: Ticker FIS
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
82 di 500: Ticker FTDR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
83 di 500: Ticker BMA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
84 di 500: Ticker ABG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
85 di 500: Ticker CTRE
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
8

...

single positional indexer is out-of-bounds
435 di 500: Ticker DXLG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
436 di 500: Ticker DGII
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
437 di 500: Ticker EEFT
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
438 di 500: Ticker JHG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
439 di 500: Ticker STRO
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
440 di 500: Ticker AVDX
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
441 di 500: Ticker CG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
442 di 500: Ticker TU
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
443 di 500: Ticker TBBK
Download dati ticker
Calcolo indicatori ticker
Definizione features e targ


1 Failed download:
['LEN^B']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
472 di 500: Ticker UVSP
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
473 di 500: Ticker MMI
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
474 di 500: Ticker AMBC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
475 di 500: Ticker LLYVK
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
index -1 is out of bounds for axis 0 with size 0
476 di 500: Ticker MSA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
477 di 500: Ticker CAT
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
478 di 500: Ticker LECO
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
479 di 500: Ticker TPRO
Download dati ticker



1 Failed download:
['TPRO']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
480 di 500: Ticker COOP
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
481 di 500: Ticker PFC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
482 di 500: Ticker UNH
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
483 di 500: Ticker HSY
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
484 di 500: Ticker LXFR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
485 di 500: Ticker SBOW
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
486 di 500: Ticker VVX
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
487 di 500: Ticker LBTYA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
488 di 500: Ticker NU
Download dati ticker
Calcolo indicatori ticker
Definizione features e targ

In [3]:
def rd(hypermodel, X_train, Y_train, X_val, Y_val):
    random_tuner = RandomSearch(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=2,
        directory='random_search',
        project_name='random_tuning'
    )
    random_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    random_tuner.search(X_train, Y_train, epochs=epochs, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    # Ottieni il miglior modello e i migliori iperparametri
    best_model_random = random_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_random = random_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Random Search:", best_hyperparameters_random.values)
    
    return best_model_random, best_hyperparameters_random

In [5]:
def bay(hypermodel, X_train, Y_train, X_val, Y_val):
    bayesian_tuner = BayesianOptimization(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        seed=seed,
        directory='bayesian_search',
        project_name='bayesian_tuning'
    )
    
    bayesian_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    bayesian_tuner.search(X_train, Y_train, epochs=epochs, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_bayesian = bayesian_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_bayesian = bayesian_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Bayesian Optimization:", best_hyperparameters_bayesian.values)
    
    return best_model_bayesian, best_hyperparameters_bayesian


In [4]:
def hb(hypermodel, X_train, Y_train, X_val, Y_val):
    hyperband_tuner = Hyperband(
        hypermodel,
        objective='val_loss',
        max_epochs=30,
        seed=seed,
        directory='hyperband_search',
        project_name='hyperband_tuning'
    )
    
    hyperband_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    hyperband_tuner.search(X_train, Y_train, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_hyperband = hyperband_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_hyperband = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Hyperband:", best_hyperparameters_hyperband.values)
    
    return best_model_hyperband, best_hyperparameters_hyperband


In [6]:
model, par = rd(hypermodel, X, Y, X_val, Y_val)

Trial 10 Complete [00h 01m 45s]
val_loss: 100.28850173950195

Best val_loss So Far: 73.93067169189453
Total elapsed time: 00h 31m 27s
INFO:tensorflow:Oracle triggered exit
Migliori iperparametri per Random Search: {'lstm_units_1': 70, 'num_lstm_layers': 3, 'lstm_units_2': 60, 'l2_rate': 0.09049148977414222, 'lstm_dropout_2': 0.30000000000000004, 'num_dense_layers': 3, 'dense_units_0': 110, 'dense_dropout_0': 0.30000000000000004, 'optimizer': 'adam', 'lstm_units_3': 20, 'lstm_dropout_3': 0.2, 'lstm_units_4': 110, 'lstm_dropout_4': 0.1, 'dense_units_1': 20, 'dense_dropout_1': 0.1, 'dense_units_2': 80, 'dense_dropout_2': 0.0}


In [7]:
model, par = bay(hypermodel, X, Y, X_val, Y_val)

Trial 9 Complete [00h 04m 11s]
val_loss: 82.947509765625

Best val_loss So Far: 67.51394653320312
Total elapsed time: 00h 49m 54s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
150               |40                |lstm_units_1
1                 |3                 |num_lstm_layers
90                |100               |lstm_units_2
0.00089355        |0.045141          |l2_rate
0.3               |0                 |lstm_dropout_2
3                 |3                 |num_dense_layers
60                |10                |dense_units_0
0                 |0.2               |dense_dropout_0
rmsprop           |adam              |optimizer
10                |50                |lstm_units_3
0.2               |0.2               |lstm_dropout_3
20                |140               |dense_units_1
0                 |0.2               |dense_dropout_1
30                |80                |dense_units_2
0.1               |0.2               |dense_dropout_2
120      